In [30]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *
import xgboost as xgb

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

X_train=X_train.drop(["DE_NET_IMPORT","FR_NET_IMPORT","DE_FR_EXCHANGE",'DAY_ID'],axis=1)

country_mapping = {'FR': 0, 'DE': 1}
X_train['COUNTRY'] = X_train['COUNTRY'].map(country_mapping)

X_train.fillna(X_train.mean(),inplace=True)
features_columns=X_train.drop('ID',axis=1).columns

In [ ]:
df=create_pairwise_dataset(Y_train)
print(df)


In [39]:
from sklearn.datasets import make_classification
import numpy as np

import xgboost as xgb

# Make a synthetic ranking dataset for demonstration
seed = 1994
X, y = make_classification(random_state=seed)
print(X.shape)
rng = np.random.default_rng(seed)
n_query_groups = 3
qid = rng.integers(0, 3, size=X.shape[0])

# Sort the inputs based on query index
sorted_idx = np.argsort(qid)
X = X[sorted_idx, :]
y = y[sorted_idx]

print(y.shape)

X_train=pd.DataFrame(X)
Y_train=pd.DataFrame(y)
X_train['qid']=qid
Y_train['qid']=qid

(100, 20)
(100,)


array([1, 1, 2, 0, 1, 1, 2, 2, 2, 0, 1, 1, 2, 0, 1, 1, 2, 1, 0, 2, 1, 1,
       2, 0, 1, 2, 2, 2, 1, 0, 0, 1, 0, 1, 0, 0, 2, 1, 2, 2, 0, 1, 1, 1,
       2, 1, 1, 1, 0, 2, 2, 2, 2, 1, 0, 0, 2, 2, 0, 2, 0, 1, 2, 0, 2, 2,
       0, 0, 1, 2, 0, 2, 2, 1, 2, 1, 0, 1, 1, 2, 1, 1, 2, 1, 2, 0, 1, 2,
       1, 2, 1, 0, 2, 2, 0, 1, 2, 0, 2, 1])

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [46]:
query_train = [X_train.shape[0]]
query_val = [X_val.shape[0]]
query_test = [X_test.shape[0]]

In [48]:
import lightgbm as lgb
gbm = lgb.LGBMRanker()
gbm.fit(X_train, y_train, group=query_train,
        eval_set=[(X_val, y_val)], eval_group=[query_val],
        eval_at=[5, 10, 20])

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 460
[LightGBM] [Info] Number of data points in the train set: 64, number of used features: 20
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

LGBMRanker()

In [49]:
import numpy as np
import pandas as pd
import lightgbm

df = pd.DataFrame({
    "query_id":[i for i in range(100) for j in range(10)],
    "var1":np.random.random(size=(1000,)),
    "var2":np.random.random(size=(1000,)),
    "var3":np.random.random(size=(1000,)),
    "relevance":list(np.random.permutation([0,0,0,0,0, 0,0,0,1,1]))*100
})

train_df = df[:800]  # first 80%
validation_df = df[800:]  # remaining 20%

qids_train = train_df.groupby("query_id")["query_id"].count().to_numpy()
X_train = train_df.drop(["query_id", "relevance"], axis=1)
y_train = train_df["relevance"]

qids_validation = validation_df.groupby("query_id")["query_id"].count().to_numpy()
X_validation = validation_df.drop(["query_id", "relevance"], axis=1)
y_validation = validation_df["relevance"]

In [50]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
)

In [52]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at=10)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005404 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 3
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMRanker(metric='ndcg', objective='lambdarank')